In [101]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [102]:
df = pd.read_csv("D:/Code/py_code/Artificial-Neural-Network/Single-Layer-Perceptron/data/train_data_cl.csv", header=None, names=list(range(13)))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NaN
1,1,0,3,Braund,Mr. Owen Harris,male,22,1,0,A/5 21171,7.25,NaN,S
2,2,1,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),female,38,1,0,PC 17599,712.833,C85,C
3,3,1,3,Heikkinen,Miss. Laina,female,26,0,0,STON/O2. 3101282,7.925,NaN,S
4,4,1,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),female,35,1,0,113803,53.1,C123,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,886,0,3,Rice,Mrs. William (Margaret Norton),female,39,0,5,382652,29.125,NaN,Q
709,887,0,2,Montvila,Rev. Juozas,male,27,0,0,211536,13,NaN,S
710,888,1,1,Graham,Miss. Margaret Edith,female,19,0,0,112053,30,B42,S
711,890,1,1,Behr,Mr. Karl Howell,male,26,0,0,111369,30,C148,C


# Data Preprocessing

### Benerin column

In [103]:
col_name = df.iloc[0,:12].values
col_name = np.insert(col_name, 4, "FirstName")

df.columns = col_name
df = df.drop(0).reset_index(drop=True)
df

,PassengerId,Survived,Pclass,Name,FirstName,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund,Mr. Owen Harris,male,22,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),female,38,1,0,PC 17599,712.833,C85,C
2,3,1,3,Heikkinen,Miss. Laina,female,26,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),female,35,1,0,113803,53.1,C123,S
4,5,0,3,Allen,Mr. William Henry,male,35,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,886,0,3,Rice,Mrs. William (Margaret Norton),female,39,0,5,382652,29.125,NaN,Q
708,887,0,2,Montvila,Rev. Juozas,male,27,0,0,211536,13,NaN,S
709,888,1,1,Graham,Miss. Margaret Edith,female,19,0,0,112053,30,B42,S
710,890,1,1,Behr,Mr. Karl Howell,male,26,0,0,111369,30,C148,C


### Merge Name dan FirstName

In [104]:
df["Name"] = df["FirstName"] + " " + df["Name"]
df = df.drop("FirstName", axis=1)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr. Owen Harris Braund,male,22,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cu...,female,38,1,0,PC 17599,712.833,C85,C
2,3,1,3,Miss. Laina Heikkinen,female,26,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,113803,53.1,C123,S
4,5,0,3,Mr. William Henry Allen,male,35,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
707,886,0,3,Mrs. William (Margaret Norton) Rice,female,39,0,5,382652,29.125,NaN,Q
708,887,0,2,Rev. Juozas Montvila,male,27,0,0,211536,13,NaN,S
709,888,1,1,Miss. Margaret Edith Graham,female,19,0,0,112053,30,B42,S
710,890,1,1,Mr. Karl Howell Behr,male,26,0,0,111369,30,C148,C


### Change Data Type

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  712 non-null    object
 1   Survived     712 non-null    object
 2   Pclass       712 non-null    object
 3   Name         712 non-null    object
 4   Sex          712 non-null    object
 5   Age          712 non-null    object
 6   SibSp        712 non-null    object
 7   Parch        712 non-null    object
 8   Ticket       712 non-null    object
 9   Fare         685 non-null    object
 10  Cabin        210 non-null    object
 11  Embarked     677 non-null    object
dtypes: object(12)
memory usage: 66.9+ KB


In [106]:
df = df.astype({'PassengerId': 'int64', 'Survived': 'int64', 'Pclass': 'int64', 'SibSp': 'int64'})
df = df.astype({'Name': 'string', 'Sex': 'string', 'Ticket': 'string', 'Cabin': 'string', 'Embarked': 'string'})
df = df.astype({'Age': 'float64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    string 
 4   Sex          712 non-null    string 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    object 
 8   Ticket       712 non-null    string 
 9   Fare         685 non-null    object 
 10  Cabin        210 non-null    string 
 11  Embarked     677 non-null    string 
dtypes: float64(1), int64(4), object(2), string(5)
memory usage: 66.9+ KB


### Menghilangkan Column yang tidak dipakai

In [107]:
del df['Name']
del df['Ticket']
del df['Cabin']

del df['Parch']
del df['Fare']


### Cek & Replace Missing Value

In [108]:
missing = pd.DataFrame({
    'total' : df.isnull().sum(),
    'percent' : df.isnull().sum()/df.shape[0] * 100})
    
missing

,total,percent
PassengerId,0,0.00000
Survived,0,0.00000
Pclass,0,0.00000
Sex,0,0.00000
Age,0,0.00000
SibSp,0,0.00000
Embarked,35,4.91573


In [109]:
df['Embarked'] = df['Embarked'].fillna(value='S')
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Embarked       0
dtype: int64

### Encode string value to int

In [114]:
lbenc = LabelEncoder()
print(df["Sex"].unique().tolist())
print(df["Embarked"].unique().tolist())

['male', 'female', '15', '27', '9', '36.5', '16', '40', '45', '24', '18', '20.5', '8', '26', '3', '1', '19', '21', '36', '22', '48', '49', '35', '39', '6']
['S', 'C', 'Q']
